In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from analysis.plotutility import *
from config.selectionconfig import plotsetting as pltcfg

In [3]:
import os

In [4]:
vis = Visualizer(pltcfg)

In [5]:
vis.grepweights(output=True)

In [6]:
vis.indir

'/uscms/home/joyzhou/nobackup/output'

In [8]:
# test combine_cf()
dirname = pjoin(vis.indir, 'DYJets')
raw_cutflow = vis.combine_cf(dirname, 'DYJetsToLL_M-10to50', output=True)
raw_cutflow

,DYJetsToLL_M-10to50
initial,22445405
ElectronSelection,22314144
MuonSelection,21925141
TauSelection,277


In [7]:
# test compute_allcf()
raw_df, wgt_df = vis.compute_allcf()

In [8]:
raw_df

,DYJetsToLL_M-10to50,DYJetsToLL_M-50,ZH_HToBB_ZToLL_M-125,ZH_HToBB_ZToQQ_M-125,ttHto2B_M-125,VBFHToTauTau_M125,GluGluHToTauTau_M-125
initial,22445405,92312235,2062940,4092774,4107465,298000,291000
ElectronSelection,22314144,68892058,1236115,3577235,2431611,221865,242886
MuonSelection,21925141,47130535,609499,3373217,1683102,187064,213728
TauSelection,277,72801,11019,9159,26490,9067,5214


In [21]:
df = load_roots('/uscms/home/joyzhou/nobackup/output/DYJets', 'DYJetsToLL_M-50_*.root', ['Electron_pt'], 'tree', clean=True)

Deleted /uscms/home/joyzhou/nobackup/output/DYJets/DYJetsToLL_M-50_tauhtauh_101-part0.root
Deleted /uscms/home/joyzhou/nobackup/output/DYJets/DYJetsToLL_M-50_tauhtauh_101-part1.root
